In [1]:
import numpy as np 
import pandas as pd

In [2]:
work_path = ''
train_order = pd.read_csv(work_path + 'train_order_add_negasmpl.gz', compression='gzip')
train_order = train_order.loc[train_order.rating == 1]
df_deal_vec = pd.read_csv('', compression='gzip')

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df_deal_vec = df_deal_vec[['deal_id', 'dealvec']]
df_deal_vec.dealvec = df_deal_vec.dealvec.map(lambda x: x[1:-1].split())

In [12]:
train_order.deal_id = train_order.deal_id.map(lambda x: str(x))
grouped_order = train_order.groupby('account_id').apply(lambda x: ' '.join(list(x['deal_id'])))
Input_Data = pd.DataFrame(grouped_order)
Input_Data = Input_Data.reset_index(drop = False)
Input_Data.columns = ['account_id', 'deal_id']
Input_Data['deal_id'] = Input_Data.deal_id.map(lambda x: x.split(' '))

In [13]:
MAX_TIME_STEP = train_order.account_id.value_counts().max()
MAX_TRAING_EXAMPLE_NUMBER = Input_Data.shape[0]

In [14]:
def padding_to_match_time_step(session):
    add_zeros = []
    for i in range(MAX_TIME_STEP - len(session)):
        add_zeros.append('0')
    session.extend(add_zeros)
    return session

In [15]:
Input_Data['deal_id'] = Input_Data.deal_id.map(lambda x: padding_to_match_time_step(x))

In [16]:
user_numpy = Input_Data.values

In [26]:
(train_order.account_id.value_counts() >= 10).sum() / (train_order.account_id.value_counts() >= 1).sum() *100

11.681003540619791

In [28]:
(train_order.account_id.value_counts() <= 50).sum()

326746

In [10]:
for element in range(len(user_numpy)):
    for deal in range(len(user_numpy[element][1])):
        if user_numpy[element][1][deal] != '0':
            user_numpy[element][1][deal] = df_deal_vec.loc[df_deal_vec.deal_id == int(user_numpy[element][1][deal])].dealvec.values[0]
        if user_numpy[element][1][deal] == '0':
            user_numpy[element][1][deal] = np.zeros(200).tolist()

KeyboardInterrupt: 

In [ ]:
np.save('deal_session_per_user.npy', user_numpy)